# Module 0 — MPC in a nutshell

### 0.1 Discrete-time model

We’ll use a linear time-invariant (LTI) model (obtained by linearizing the inverted pendulum in Module 1 and sampling with zero-order hold):

$x_{k+1}=A x_k + B u_k,\qquad x_k\in\mathbb{R}^{n},\;u_k\in\mathbb{R}^{m}$,

with $x_0$ measured at each control step.

### 0.2 Finite-horizon MPC objective

Given a prediction horizon $N$, references $\{x_k^{\text{ref}}\}_{k=0}^N$ and (optional) $\{u_k^{\text{ref}}\}_{k=0}^{N-1}$, and weights $Q\succeq 0$, $Q_f\succeq 0$, $R\succ 0$,
MPC minimizes:

$$\min_{\{x_k,u_k\}}
\sum_{k=0}^{N-1}
\Big[(x_k-x_k^{\text{ref}})^\top Q (x_k-x_k^{\text{ref}}) + (u_k-u_k^{\text{ref}})^\top R (u_k-u_k^{\text{ref}})\Big]
	+ (x_N-x_N^{\text{ref}})^\top Q_f (x_N-x_N^{\text{ref}}).
$$

### 0.3 Constraints

$$
x_k \in \mathcal X = \{x:\; C_x x \le d_x\},\qquad
u_k \in \mathcal U = \{u:\; C_u u \le d_u\}.
$$
For the inverse pendulum case, we’ll also allow optional soft angle bounds via slacks $s_k\ge 0$ (Module 2), penalized quadratically: $\lambda_s\sum\|s_k\|_2^2$.

### 0.4 “Condensed” QP (eliminate states)

Stack the variables:
$X \!=\! \begin{bmatrix}x_1\\ \vdots \\ x_N\end{bmatrix},\quad
U \!=\! \begin{bmatrix}u_0\\ \vdots \\ u_{N-1}\end{bmatrix}$.
By repeated substitution,
$X = S_x\,x_0 + S_u\,U$,
where $S_x$ and $S_u$ are the standard prediction matrices:
$$
x_k = A^k x_0 + \sum_{j=0}^{k-1} A^{k-1-j} B\,u_j,\quad k=1,\dots,N.
$$
Define block weights

$$
\bar Q=\mathrm{blkdiag}(
	
	\underbrace{Q,\dots,Q}_{N-1\text{ times}}
	
	,Q_f),\quad
\bar R=\mathrm{blkdiag}(
	
	\underbrace{R,\dots,R}_{N\text{ times}}
	
	),
$$

and references 
$$
\bar X^{\text{ref}}=\mathrm{vec}(x_1^{\text{ref}},\dots,x_N^{\text{ref}}), \bar U^{\text{ref}}=\mathrm{vec}(u_0^{\text{ref}},\dots,u_{N-1}^{\text{ref}}).
$$
The condensed quadratic program becomes
$$
\min_U \;\frac12 U^\top H\,U + f(x_0)^\top U + \text{const}
$$
subject to linear inequalities
$$
G\,U \le h(x_0),
$$
with
$$
H = S_u^\top \bar Q S_u + \bar R,\qquad
f(x_0)= S_u^\top \bar Q (S_x x_0 - \bar X^{\text{ref}}) - \bar R \bar U^{\text{ref}},
$$
and, for constraints,
$$

\underbrace{\mathrm{blkdiag}(C_x,\dots,C_x)}_{\bar C_x}

 X \le \bar d_x
\;\Rightarrow\;
\bar C_x S_u\,U \le \bar d_x - \bar C_x S_x x_0, \quad

\underbrace{\mathrm{blkdiag}(C_u,\dots,C_u)}_{\bar C_u}\,U \le \bar d_u.

$$
This condensed form is standard and critical for fast MPC since only the vectors $f(x_0)$ and $h(x_0)$ change online; the sparsity pattern of $H$,$G$ stays fixed and can be factorized/cached by the solver.  ￼

### 0.5 How we’ll solve it (OSQP)

We map to OSQP’s form
$$
\min_{z}\; \tfrac12 z^\top P z + q^\top z \quad \text{s.t.}\quad \ell \le A z \le u,
$$
with $z=U$ (or $z=[U;s]$ if we include soft-constraint slacks). OSQP is an operator-splitting (ADMM) QP solver that factorizes a quasi-definite KKT matrix once, then performs cheap iterations, supports warm starts, detects infeasibility, and excels for real-time MPC with repeated solves and fixed sparsity.  ￼ ￼

⸻

### 0.6 Python setup (shared utilities for all modules)

This cell sets global knobs you asked to keep handy and builds reusable prediction/cost/constraint matrices for the condensed QP. (We’ll plug in the cart-pole $A$,$B$ in Module 1 and the task-specific constraints in Module 2, then call OSQP in Module 3.)

In [ ]:
# === Module 0: Core MPC utilities (NumPy/SciPy/OSQP) ===
from __future__ import annotations

import numpy as np
import scipy.sparse as sp

# ----------------------------
# Global handles (easy to tune)
# ----------------------------
DT = 0.02          # sampling time [s]
N_HORIZON = 30     # prediction steps

# Weights (defaults; cart-pole-specific values will be set in Module 2)
def default_weights(n_x, n_u):
    Q  = np.diag([1.0]*n_x)           # state stage cost
    R  = 1e-2 * np.eye(n_u)           # input stage cost
    Qf = 10.0 * Q                     # terminal cost
    return Q, R, Qf

# ----------------------------
# Prediction matrices Sx, Su
# ----------------------------
def build_prediction_matrices(A, B, N):
    """Return (Sx, Su) for X = Sx x0 + Su U with horizon N.
    X stacks x_1..x_N, U stacks u_0..u_{N-1}.
    Shapes:
      Sx: (N*n_x, n_x), Su: (N*n_x, N*n_u)
    """
    A = np.asarray(A)
    B = np.asarray(B)
    n_x, n_u = A.shape[0], B.shape[1]

    # Powers of A
    A_powers = [np.eye(n_x)]
    for k in range(1, N+1):
        A_powers.append(A_powers[-1] @ A)

    # Build Sx
    blocks_Sx = [A_powers[k] for k in range(1, N+1)]
    Sx = np.vstack(blocks_Sx)

    # Build Su as block lower-triangular with A^{k-1-j} B in (k,j) for j < k
    Su = np.zeros((N*n_x, N*n_u))
    for k in range(1, N+1):
        for j in range(0, k):
            Akj = A_powers[k-1-j]
            Su[(k-1)*n_x:k*n_x, j*n_u:(j+1)*n_u] = Akj @ B

    return Sx, Su

# ----------------------------
# Block-diagonal helpers
# ----------------------------
def blkdiag_repeat(M, times):
    """Return sparse blockdiag of M repeated `times` times."""
    return sp.block_diag([sp.csr_matrix(M) for _ in range(times)], format="csr")

def stack_refs(refs):
    """Stack a list/array of per-step references into a single column vector."""
    return np.vstack([r.reshape(-1,1) for r in refs])

# ----------------------------
# Condensed cost (H, f)
# ----------------------------
def build_condensed_cost(A, B, Q, R, Qf, N, x0, x_refs=None, u_refs=None):
    """
    Return H (sparse), f (dense), plus Sx, Su for later reuse.
    x_refs: list/array of length N with n_x each (x_1..x_N references)
    u_refs: list/array of length N with n_u each (u_0..u_{N-1} references)
    """
    n_x, n_u = A.shape[0], B.shape[1]
    Sx, Su = build_prediction_matrices(A, B, N)

    # Big Q and R
    Qbar = blkdiag_repeat(Q, N-1)
    Qbar = sp.block_diag([Qbar, sp.csr_matrix(Qf)], format="csr")  # add terminal
    Rbar = blkdiag_repeat(R, N)

    # References
    if x_refs is None:
        x_refs = [np.zeros(n_x) for _ in range(N)]
    if u_refs is None:
        u_refs = [np.zeros(n_u) for _ in range(N)]
    Xref = stack_refs(x_refs)    # (N*n_x, 1)
    Uref = stack_refs(u_refs)    # (N*n_u, 1)

    # Hessian and gradient
    H = (Su.T @ Qbar @ Su) + Rbar
    # f(U) = U^T [ Su^T Qbar (Sx x0 - Xref)  - Rbar Uref ]  (no 1/2 on linear term)
    f = (Su.T @ (Qbar @ (Sx @ x0.reshape(-1,1) - Xref)) - Rbar @ Uref).ravel()

    return H.tocsc(), f, Sx, Su, Qbar, Rbar

# ----------------------------
# Condensed constraints G U <= h(x0)
# ----------------------------
def build_condensed_inequalities(A, B, N, Cx=None, dx=None, Cu=None, du=None, Sx=None, Su=None, x0=None):
    """
    Build (G, h) with state and input linear inequalities.
    If Cx,dx provided: apply to each x_k (k=1..N).
    If Cu,du provided: apply to each u_k (k=0..N-1).
    """
    n_x, n_u = A.shape[0], B.shape[1]
    if Sx is None or Su is None:
        Sx, Su = build_prediction_matrices(A, B, N)

    G_list, h_list = [], []

    # State constraints
    if Cx is not None and dx is not None:
        Cx = sp.csr_matrix(Cx)
        Cx_bar = blkdiag_repeat(Cx, N)                           # (N*n_cx, N*n_x)
        Gx = Cx_bar @ Su                                         # (N*n_cx, N*n_u)
        if x0 is None:
            raise ValueError("x0 required to build state-constraint offsets h(x0).")
        hx = np.repeat(dx.reshape(-1,1), N, axis=0) - (Cx_bar @ (Sx @ x0.reshape(-1,1)))
        G_list.append(Gx)
        h_list.append(hx)

    # Input constraints
    if Cu is not None and du is not None:
        Cu = sp.csr_matrix(Cu)
        Cu_bar = blkdiag_repeat(Cu, N)                           # (N*n_cu, N*n_u)
        Gu = Cu_bar                                              # directly on U
        hu = np.repeat(du.reshape(-1,1), N, axis=0)
        G_list.append(Gu)
        h_list.append(hu)

    if G_list:
        G = sp.vstack(G_list, format="csr")
        h = np.vstack(h_list).ravel()
    else:
        G = sp.csr_matrix((0, N*n_u))
        h = np.zeros((0,))
    return G.tocsc(), h

# Module 1 — Inverted Pendulum Dynamics

### 1.1 States, input, and parameters

We consider a cart–pole (inverted pendulum on a cart) with:

* State $x = [\,p,\ \dot p,\ \theta,\ \dot\theta\,]^\top \in \mathbb{R}^4$, where $p$ is cart position along a horizontal track, and $\theta$ is the pole angle measured from upright ($\theta=0$ means perfectly upright).
* Control input $u\in\mathbb{R}$: horizontal force applied to the cart (to the right is positive).
* Physical parameters (all SI):
	* $M$: cart mass, $m$: pole mass,
	* $l$: distance from pivot to the pole’s center of mass,
	* $I$: pole moment of inertia about its center of mass,
	* $g$: gravitational acceleration,
	* We assume zero friction.

Let $q = [\,p,\ \theta\,]^\top$. The dynamics can be written in the standard manipulator form
$$
M(q)\,\ddot q + C(q,\dot q) + G(q) \;=\; B\,u,
$$
with
$$
M(q) =
\begin{bmatrix}
M+m & m\,l\,\cos\theta\\
m\,l\,\cos\theta & I + m\,l^2
\end{bmatrix},\qquad
C(q,\dot q) =
\begin{bmatrix}
-\,m\,l\,\sin\theta\,\dot\theta^2\\[2mm]
0
\end{bmatrix},\qquad
G(q) =
\begin{bmatrix}
0\\[1mm]
-\;m\,g\,l\,\sin\theta
\end{bmatrix},
\qquad
B=\begin{bmatrix}1\\[1mm]0\end{bmatrix}.
$$

Notes:
* The centripetal term $-m l \sin\theta\,\dot\theta^2$ appears in the cart equation.
* The gravity vector is written so that $\theta=0$ is an unstable equilibrium (positive $\theta$ produces a torque that increases $\theta$).

Solving for accelerations yields
$$
\begin{bmatrix}\ddot p\\ \ddot\theta\end{bmatrix}
\;=\;
M(q)^{-1}
\left(
\underbrace{\begin{bmatrix}u\\ 0\end{bmatrix}}_{\text{input}}

\underbrace{C(q,\dot q)}_{\text{centripetal}}

\underbrace{G(q)}_{\text{gravity}}
\right).
$$
Therefore the continuous-time state-space model is
$$
\dot x \;=\; f(x,u)\;=\;
\begin{bmatrix}
\dot p\\[1mm]
\ddot p(x,u)\\[1mm]
\dot\theta\\[1mm]
\ddot\theta(x,u)
\end{bmatrix}.
$$

### 1.2 Linearization about the upright equilibrium

We linearize the frictionless model about the equilibrium
$$
x^\star = \begin{bmatrix}0\\0\\0\\0\end{bmatrix},\qquad u^\star=0.
$$
Let 
$$A_c = \left.\frac{\partial f}{\partial x}\right|{(x^\star,u^\star)}, B_c=\left.\frac{\partial f}{\partial u}\right|{(x^\star,u^\star)}.$$
Define the constant
$$
\Delta \;=\; (M+m)\,(I+m l^2) - (m l)^2 \;>\; 0.
$$
A convenient and compact way to write the linearized model is
$$
\dot x = A_c\,x + B_c\,u,\qquad
A_c=
\begin{bmatrix}
0 & 1 & 0 & 0\\
0 & 0 & a_{23} & 0\\
0 & 0 & 0 & 1\\
0 & 0 & a_{43} & 0
\end{bmatrix},\qquad
B_c=
\begin{bmatrix}
0\\ b_{2}\\ 0\\ b_{4}
\end{bmatrix},
$$
with the coefficients (valid for $\theta=0$)
$$
a_{23} = -\,\frac{m^2 l^2 g}{\Delta},\qquad
a_{43} = \frac{(M+m)\,m\,g\,l}{\Delta},\qquad
b_{2} = \frac{I + m l^2}{\Delta},\qquad
b_{4} = -\,\frac{m l}{\Delta}.
$$
Special case — point mass at the end of a massless pole ($I=0$):
$$
\Delta = M\,m\,l^2,\quad
a_{23} = -\frac{m g}{M},\quad
a_{43} = \frac{(M+m) g}{M l},\quad
b_{2} = \frac{1}{M},\quad
b_{4} = -\frac{1}{M l}.
$$

### 1.3 Discrete-time (exact ZOH discretization)

For a sampling period $\Delta t$, the exact zero-order-hold discretization is
$$
A = \exp(A_c\,\Delta t),\qquad
B = \left(\int_{0}^{\Delta t} \exp(A_c\,\tau)\,d\tau\right) B_c.
$$
Numerically, this is computed robustly via the standard block-matrix exponential:
$$
\exp\!\left(
\begin{bmatrix}
A_c & B_c\\
0 & 0
\end{bmatrix}
\Delta t\right)
=
\begin{bmatrix}
A & B\\
0 & I
\end{bmatrix}.
$$

We’ll implement this with ```scipy.linalg.expm```. These $A$,$B$ plug directly into the Module 0 utilities and (later) into the OSQP-based MPC (Module 3).

⸻

### 1.4 Module 1 — Implementation (Code cell)

In [ ]:
# === Module 1: Inverted pendulum model (frictionless) ===
from dataclasses import dataclass
import numpy as np
from numpy.typing import ArrayLike
from scipy.linalg import expm

# --------------------------------------------------------
# Parameters (all SI). Defaults match our earlier plan.
# --------------------------------------------------------
@dataclass
class CartPoleParams:
    M: float = 0.5     # cart mass [kg]
    m: float = 0.2     # pole mass [kg]
    l: float = 0.3     # COM distance from pivot [m]
    I: float = None    # pole inertia about COM [kg·m^2]; default = (1/3) m l^2 (slender rod of length 2l)
    g: float = 9.81    # gravity [m/s^2]

    def __post_init__(self):
        if self.I is None:
            # Slender rod of full length 2l: I_com = (1/12) m (2l)^2 = (1/3) m l^2
            self.I = (1.0/3.0) * self.m * (self.l**2)

# --------------------------------------------------------
# Nonlinear continuous-time dynamics: xdot = f(x,u)
# State x = [p, pdot, theta, thetadot]; input u = horizontal force
# --------------------------------------------------------
def cartpole_rhs(x: ArrayLike, u: float, par: CartPoleParams) -> np.ndarray:
    p, pdot, th, thdot = np.asarray(x, dtype=float).ravel()
    M, m, l, I, g = par.M, par.m, par.l, par.I, par.g

    s, c = np.sin(th), np.cos(th)

    # Mass matrix
    M11 = M + m
    M12 = m * l * c
    M21 = M12
    M22 = I + m * l**2
    det = M11 * M22 - M12 * M21

    # Bias terms (centripetal + gravity) written as C + G
    C1 = - m * l * s * thdot**2
    C2 = 0.0
    G1 = 0.0
    G2 = - m * g * l * s  # chosen so that theta=0 is an unstable equilibrium

    # Right-hand side: Bu - (C + G)
    r1 = u - (C1 + G1)
    r2 = 0.0 - (C2 + G2)

    # qddot = M^{-1} r
    inv11 =  M22 / det
    inv12 = -M12 / det
    inv21 = -M21 / det
    inv22 =  M11 / det

    pddot   = inv11 * r1 + inv12 * r2
    thddot  = inv21 * r1 + inv22 * r2

    return np.array([pdot, pddot, thdot, thddot], dtype=float)

# --------------------------------------------------------
# Linearization at the upright equilibrium (analytic closed form)
# --------------------------------------------------------
def linearize_cartpole_upright(par: CartPoleParams):
    M, m, l, I, g = par.M, par.m, par.l, par.I, par.g
    Delta = (M + m) * (I + m * l**2) - (m * l)**2

    a23 = - (m**2) * (l**2) * g / Delta
    a43 =   (M + m) * m * g * l     / Delta
    b2  =   (I + m * l**2)          / Delta
    b4  = - (m * l)                 / Delta

    Ac = np.array([[0, 1, 0, 0],
                   [0, 0, a23, 0],
                   [0, 0, 0, 1],
                   [0, 0, a43, 0]], dtype=float)
    Bc = np.array([[0.0],
                   [b2],
                   [0.0],
                   [b4]], dtype=float)
    return Ac, Bc

# --------------------------------------------------------
# Generic numeric linearization (central differences)
# Useful as a sanity check and for other equilibria if needed.
# --------------------------------------------------------
def numeric_linearize(rhs_fun, x_eq, u_eq, par: CartPoleParams, eps: float = 1e-6):
    x_eq = np.asarray(x_eq, dtype=float).ravel()
    n = x_eq.size
    # A = df/dx
    A = np.zeros((n, n), dtype=float)
    for i in range(n):
        dx = np.zeros_like(x_eq)
        dx[i] = eps
        f_plus  = rhs_fun(x_eq + dx, u_eq, par)
        f_minus = rhs_fun(x_eq - dx, u_eq, par)
        A[:, i] = (f_plus - f_minus) / (2 * eps)

    # B = df/du
    du = eps
    f_plus  = rhs_fun(x_eq, u_eq + du, par)
    f_minus = rhs_fun(x_eq, u_eq - du, par)
    B = ((f_plus - f_minus) / (2 * du)).reshape(n, 1)
    return A, B

# --------------------------------------------------------
# Continuous-to-discrete (exact ZOH via augmented matrix exponential)
# --------------------------------------------------------
def c2d_exact(Ac: ArrayLike, Bc: ArrayLike, dt: float):
    Ac = np.asarray(Ac, dtype=float)
    Bc = np.asarray(Bc, dtype=float)
    n  = Ac.shape[0]
    aug = np.zeros((n + Bc.shape[1], n + Bc.shape[1]), dtype=float)
    aug[:n, :n] = Ac
    aug[:n, n:] = Bc
    # expm of block matrix
    E = expm(aug * dt)
    Ad = E[:n, :n]
    Bd = E[:n, n:]
    return Ad, Bd

# --------------------------------------------------------
# Convenience: build (A, B) for MPC at the chosen dt (DT from Module 0)
# --------------------------------------------------------
def build_discrete_linear_cartpole(par: CartPoleParams, dt: float):
    Ac, Bc = linearize_cartpole_upright(par)
    A, B   = c2d_exact(Ac, Bc, dt)
    return A, B, Ac, Bc

# Module 2 — Loss and Constraints for the Inverted Pendulum

### 2.1 Quadratic loss for cart–pole regulation

We stabilize the upright equilibrium $x^\star=0$, $u^\star=0$ with a standard finite-horizon quadratic cost:
$$
J\;=\; \sum_{k=0}^{N-1}\big[(x_k-x_k^{\text{ref}})^\top Q(x_k-x_k^{\text{ref}})
+(u_k-u_k^{\text{ref}})^\top R(u_k-u_k^{\text{ref}})\big]
+(x_N-x_N^{\text{ref}})^\top Q_f (x_N-x_N^{\text{ref}}),
$$
where $x=[p,\ \dot p,\ \theta,\ \dot\theta]^\top$. For regulation we take
$$x_k^{\text{ref}}=\mathbf{0},\qquad u_k^{\text{ref}}=0.$$

Recommended default weights (tunable):
$$
Q=\mathrm{diag}(1,\ 0.1,\ 50,\ 5),\quad
R=0.01,\quad
Q_f=10\,Q.
$$
This prioritizes small angular deviation while keeping inputs moderate.

### 2.2 Hard constraints (inputs and track)

We enforce realistic hard bounds:

* Input force: $|u_k|\le u_{\max}$ (e.g., $u_{\max}=10 N$).
In stacked form: 
$$
(I_N\otimes C_u)U\le \mathbf{1}_N\otimes d_u
$$ 
with
$$
C_u=\begin{bmatrix}1\\-1\end{bmatrix},\ d_u=\begin{bmatrix}u_{\max}\\u_{\max}\end{bmatrix}.
$$
* Track position: $|p_k|\le x_{\max}$ (e.g., $x_{\max}=2.4 m$).
Per state: $$C_x=\begin{bmatrix}e_p^\top\\-e_p^\top\end{bmatrix}, d_x=\begin{bmatrix}x_{\max}\\x_{\max}\end{bmatrix},$$
where $e_p=[1,0,0,0]$. Stacked: 
$$(I_N\otimes C_x)X\le \mathbf{1}_N\otimes d_x.$$

These become linear inequalities in the condensed variable $U$ using $X=S_x x_0 + S_u U$ (see Module 0).

### 2.3 Optional soft angle bound

We often want $|\theta_k|\le \theta_{\max}$ (e.g., $\theta_{\max}=0.25 rad$) but without infeasibility under disturbances.
Introduce per-step slack $s_k\ge 0$ and enforce
$$\theta_k \le \theta_{\max}+s_k,\qquad -\theta_k \le \theta_{\max}+s_k.$$
Slacks are penalized in the objective by $\lambda_s \sum_{k=1}^{N}s_k^2$ with $\lambda_s>0$.

Let $z=[U;\ s]\in\mathbb{R}^{Nm+N}$. Using $\theta_k=e_\theta^\top x_k$ with $e_\theta=[0,0,1,0]$,
$$\theta_k = e_\theta^\top (S_x^{(k)} x_0 + S_u^{(k)} U).$$
The two inequalities at each step become linear in $z$:
$$
\begin{aligned}
\ &\ e_\theta^\top S_u^{(k)}\,U - 1\cdot s_k \ \le\ \theta_{\max} - e_\theta^\top S_x^{(k)} x_0,\\
\ &-e_\theta^\top S_u^{(k)}\,U - 1\cdot s_k \ \le\ \theta_{\max} + e_\theta^\top S_x^{(k)} x_0,
\end{aligned}
$$
plus $s_k\ge 0$ for all $k$. These rows do not change sparsity across time; only their right-hand sides depend on $x_0$, which is perfect for fast OSQP updates.

### 2.4 What this module provides

* Default cart-pole weights $Q$,$R$,$Q_f$.
* Hard constraint builders for input and track limits.
* Optional soft angle builder with slacks, producing an OSQP-ready structure:
$$\min_{z} \tfrac12 z^\top P z + q^\top z\quad\text{s.t.}\quad \ell \le A z \le u,$$
where $P=\mathrm{blkdiag}(H,\ \lambda_s I)$ if slacks are enabled (otherwise $P=H$), $q$ contains the condensed linear term, and $(A,\ell,u)$ stacks all inequalities as described.

⸻

### 2.5 Module 2 — Implementation (Code cell)

In [ ]:
# === Module 2: Cart-pole loss & constraints (OSQP-ready structure) ===
from dataclasses import dataclass
import numpy as np
import scipy.sparse as sp

# Imports from Module 0 utilities
# - DT, N_HORIZON
# - build_prediction_matrices, blkdiag_repeat
# Provide fallbacks if this cell is run standalone (can be removed if Module 0 is guaranteed to run first).
try:
    DT
except NameError:
    DT = 0.02
try:
    N_HORIZON
except NameError:
    N_HORIZON = 30

def _blkdiag_repeat(M, times):
    # Fallback helper if blkdiag_repeat isn't in scope
    return sp.block_diag([sp.csr_matrix(M) for _ in range(times)], format="csr")

# ----------------------------
# Default cart-pole weights
# ----------------------------
def cartpole_default_weights():
    Q  = np.diag([1.0, 0.1, 50.0, 5.0])  # [p, p_dot, theta, theta_dot]
    R  = 1e-2 * np.eye(1)                # single-input force
    Qf = 10.0 * Q
    return Q, R, Qf

# ----------------------------
# Constraint configs
# ----------------------------
@dataclass
class SoftAngleConfig:
    enabled: bool = True
    theta_max: float = 0.25   # [rad]
    lambda_s: float = 100.0   # penalty on slack^2

@dataclass
class ConstraintConfig:
    u_max: float = 10.0       # |u| <= u_max  [N]
    x_max: float = 2.4        # |p| <= x_max  [m]
    soft_angle: SoftAngleConfig = SoftAngleConfig()

# ----------------------------
# Problem structure container
# ----------------------------
@dataclass
class CondensedQPStructure:
    # Constant across time (factorizable once by OSQP)
    P: sp.csc_matrix          # Hessian on [U; s] (or on U if no slacks)
    A: sp.csc_matrix          # Constraint matrix on [U; s]
    N: int                    # horizon
    n_x: int                  # state dimension (4)
    n_u: int                  # input dimension (1)
    nU: int                   # N * n_u
    nS: int                   # N (if soft angle enabled) else 0
    use_soft_angle: bool

    # Precomputed prediction/cost pieces
    Sx: np.ndarray
    Su: np.ndarray
    Qbar: sp.csr_matrix
    Rbar: sp.csr_matrix
    H: sp.csc_matrix          # Hessian on U only

    # Hard-constraint ingredients for vector updates
    Cx: np.ndarray | None
    dx: np.ndarray | None
    Cx_bar: sp.csr_matrix | None
    G_hard: sp.csc_matrix     # rows affecting U only
    hard_row_count: int

    # Angle selection helpers
    theta_row_idx: np.ndarray | None  # indices of theta rows in Sx/Su (length N)

    def build_vectors(self, x0, x_refs=None, u_refs=None, uref_scalar=0.0):
        """
        Build q, l, u for OSQP given current x0 and optional references.
        Returns (q, l, u), where:
          - q has size nU (+ nS if soft angle)
          - l, u have size equal to A.shape[0]
        """
        n, m, N = self.n_x, self.n_u, self.N
        x0 = np.asarray(x0).reshape(-1, 1)

        # References (defaults to zeros)
        if x_refs is None:
            Xref = np.zeros((N * n, 1))
        else:
            Xref = np.vstack([np.asarray(r).reshape(-1,1) for r in x_refs])

        if u_refs is None:
            Uref = np.full((N * m, 1), float(uref_scalar))
        else:
            Uref = np.vstack([np.asarray(r).reshape(-1,1) for r in u_refs])

        # Linear term for condensed cost on U:
        # f = Su^T Qbar (Sx x0 - Xref) - Rbar Uref
        f = (self.Su.T @ (self.Qbar @ (self.Sx @ x0 - Xref)) - self.Rbar @ Uref).ravel()

        if self.use_soft_angle:
            # q = [f; 0], since slacks only enter through P (quadratic)
            q = np.concatenate([f, np.zeros(self.nS)])
        else:
            q = f

        # Hard constraints: G_hard U <= h_hard(x0)
        h_hard_list = []
        if self.Cx_bar is not None:
            # State-position (track) bounds
            hx = np.repeat(self.dx.reshape(-1,1), N, axis=0) - (self.Cx_bar @ (self.Sx @ x0))
            h_hard_list.append(hx)
        # Input bounds (already stacked inside G_hard with Cu_bar)
        # upper bound is just du repeated
        # We constructed G_hard with both state and input rows, so we need both parts of h:
        # To keep it simple, we appended state first (if any), then input.
        if h_hard_list:
            h_state = np.vstack(h_hard_list)
            # Input piece:
            # For inputs, rows count is 2*N (for +/-), with RHS = [u_max,...,u_max]^T
            hu = np.full((2*N, 1), self.du_scalar)  # self.du_scalar set during construction
            h_hard = np.vstack([h_state, hu])
        else:
            hu = np.full((2*N, 1), self.du_scalar)
            h_hard = hu

        # OSQP uses l <= A z <= u. Our hard inequalities are of the form G z <= h.
        # So l = -inf, u = h for those rows.
        inf = np.inf
        l_list = [ -inf * np.ones((self.hard_row_count,)) ]
        u_list = [ h_hard.ravel() ]

        if self.use_soft_angle:
            # Soft angle rows:
            # Build the RHS using theta rows of Sx
            Theta_Sx_x0 = (self.Sx[self.theta_row_idx, :] @ x0).ravel()  # length N

            # Upper bounds for +theta and -theta constraints
            u_theta_pos = self.theta_max * np.ones(N) - Theta_Sx_x0
            u_theta_neg = self.theta_max * np.ones(N) + Theta_Sx_x0
            u_theta = np.concatenate([u_theta_pos, u_theta_neg])

            # Their lower bounds are -inf
            l_theta = -inf * np.ones(2 * N)

            # Nonnegativity of slacks: s >= 0
            l_s = np.zeros(N)
            u_s = inf * np.ones(N)

            l_list += [l_theta, l_s]
            u_list += [u_theta, u_s]

        l = np.concatenate(l_list)
        u = np.concatenate(u_list)
        return q, l, u

# ----------------------------
# Builder for OSQP-ready structure
# ----------------------------
def prepare_cartpole_qp_structure(A, B, Q, R, Qf, N, constraints: ConstraintConfig):
    """
    Precompute constant (P, A) and store everything needed to build (q, l, u)
    at runtime given x0 and references.
    """
    A = np.asarray(A); B = np.asarray(B)
    n_x, n_u = A.shape[0], B.shape[1]
    assert n_x == 4 and n_u == 1, "This helper is specialized for cart-pole (4 states, 1 input)."

    # Prediction matrices and cost blocks
    Sx, Su = build_prediction_matrices(A, B, N)
    Qbar = _blkdiag_repeat(Q, N-1)
    Qbar = sp.block_diag([Qbar, sp.csr_matrix(Qf)], format="csr")
    Rbar = _blkdiag_repeat(R, N)
    H = (sp.csr_matrix(Su).T @ Qbar @ sp.csr_matrix(Su)) + Rbar  # keep sparse all the way
    H = H.tocsc()

    # ---- Hard constraints on position and input ----
    e_p = np.array([[1.0, 0.0, 0.0, 0.0]])
    Cx = np.vstack([ e_p, -e_p ])
    dx = np.array([constraints.x_max, constraints.x_max], dtype=float)

    Cu = np.array([[1.0], [-1.0]])
    du = np.array([constraints.u_max, constraints.u_max], dtype=float)

    Cx_bar = _blkdiag_repeat(Cx, N)            # sparse
    Su_sp  = sp.csr_matrix(Su)                 # <-- NEW: sparse Su
    Gx     = (Cx_bar @ Su_sp).tocsc()          # stays sparse
    Cu_bar = _blkdiag_repeat(Cu, N).tocsc()

    G_hard = sp.vstack([Gx, Cu_bar], format="csc")
    hard_row_count = G_hard.shape[0]

    # ---- Optional soft angle constraints with slacks ----
    use_soft = constraints.soft_angle.enabled
    if use_soft:
        theta_max = constraints.soft_angle.theta_max
        lambda_s  = constraints.soft_angle.lambda_s
        nS = N
        theta_row_idx = np.arange(N) * n_x + 2  # theta row in stacked states

        Theta_Su = Su_sp[theta_row_idx, :]      # (N, N*n_u) sparse

        A_top       = sp.hstack([G_hard, sp.csc_matrix((hard_row_count, nS))], format="csc")
        A_theta_pos = sp.hstack([ Theta_Su, -sp.eye(nS, format="csc") ], format="csc")
        A_theta_neg = sp.hstack([-Theta_Su, -sp.eye(nS, format="csc") ], format="csc")
        A_spos      = sp.hstack([ sp.csc_matrix((nS, Theta_Su.shape[1])), sp.eye(nS, format="csc") ], format="csc")
        A = sp.vstack([A_top, A_theta_pos, A_theta_neg, A_spos], format="csc")

        P = sp.block_diag([H, lambda_s * sp.eye(nS, format="csc")], format="csc")
    else:
        theta_max = None
        lambda_s  = None
        nS = 0
        theta_row_idx = None
        A = G_hard.copy()
        P = H.copy()

    spec = CondensedQPStructure(
        P=P.tocsc(), A=A.tocsc(), N=N, n_x=n_x, n_u=n_u,
        nU=N*n_u, nS=nS, use_soft_angle=use_soft,
        Sx=Sx, Su=Su, Qbar=Qbar, Rbar=Rbar, H=H,
        Cx=Cx, dx=dx, Cx_bar=Cx_bar, G_hard=G_hard, hard_row_count=hard_row_count,
        theta_row_idx=theta_row_idx
    )
    # set attributes post-init
    spec.theta_max = theta_max
    spec.lambda_s  = lambda_s
    spec.du_scalar = float(constraints.u_max)
    return spec

# Module 3 — MPC Solver

### 3.1 From condensed QP to OSQP

From Module 0–2, our condensed problem (with optional slacks for soft angle) is
$$
\min_{z}\ \tfrac12 z^\top P z + q(x_0)^\top z
\quad\text{s.t.}\quad \ell(x_0) \le A z \le u(x_0),
$$
where
* $z = U \in \mathbb{R}^{N m}$ if no soft angle, or $z = [U;\ s]\in\mathbb{R}^{Nm+N}$ with slacks s if soft angle is enabled,
* $P$ and $A$ are constant across control steps (same sparsity ⇒ same factorization),
* only the vectors $q,\ell,u$ depend on the current measured state $x_0$ (and references).

We use OSQP, which solves QPs in the form
$$
\min_{z}\ \tfrac12 z^\top P z + q^\top z\quad\text{s.t.}\quad \ell \le A z \le u.
$$
We configure OSQP once with ($P$,$A$), then at each control step:
	1.	update $q,\ell,u$ with the current $x_0$,
	2.	warm-start with a shifted solution from the previous step,
	3.	call ```solve()```, and apply $u_k = U^\star_0$ (first element of $U^\star$).

### 3.2 Warm-start and shifting

If the previous optimal decision was $U^\star = [u_0^\star,\dots,u_{N-1}^\star]$,
a good warm start at the next step is the shifted vector
$$
\tilde U = [u_1^\star,\ u_2^\star,\ \dots,\ u_{N-1}^\star,\ u_{N-1}^\star].
$$
If slacks are present, shift them similarly: 
$$
\tilde s = [s_2^\star,\dots,s_N^\star, s_N^\star].
$$
This keeps OSQP iterates close to the new optimum and stabilizes real-time performance.

### 3.3 Pseudocode

```pseudo
Precompute spec = prepare_cartpole_qp_structure(A,B,Q,R,Qf,N,constraints)
Setup OSQP with (P=spec.P, A=spec.A, q=zeros, l=-inf, u=+inf)
z_init = zeros(N*m + (N if soft-angle else 0))

Loop (every Δt):
    # 1) Build vectors for current measured state x0
    (q, l, u) = spec.build_vectors(x0, refs)
    osqp_model.update(q=q, l=l, u=u)

    # 2) Warm-start with shift of last solution
    z_warm = shift(z_prev)
    osqp_model.warm_start(x=z_warm)

    # 3) Solve
    result = osqp_model.solve()
    z_star = result.x

    # 4) Apply first control to plant
    u_cmd = z_star[0]

    # 5) Cache z_prev = z_star for next shift
end loop
```

### 3.4 Solver tuning notes
* For this small QP (scalar input, $N\approx 30$), OSQP is typically sub-millisecond on a laptop CPU with warm-starts.
* Practical tolerances: $eps_{abs} = eps_{rel} ≈ 10^{-4}$ are more than adequate here.
* If you ever hit numerical issues, increase R slightly (input regularization), or clamp $\theta_{\max}$ softly with a larger slack penalty $\lambda_s$.

### 3.5 Module 3 - Implementation (Code cell)

In [ ]:
# === Module 3: OSQP-based MPC solver ===
import time
import numpy as np
import scipy.sparse as sp
import osqp  # pip install osqp

# Uses:
# - CondensedQPStructure, prepare_cartpole_qp_structure (Module 2)
# - DT, N_HORIZON (Module 0)

class LinearMPC:
    """
    Real-time linear MPC solver using OSQP on condensed QP.
    """
    def __init__(self, spec, osqp_settings=None):
        """
        spec: CondensedQPStructure from prepare_cartpole_qp_structure(...)
        osqp_settings: dict of OSQP options (eps_abs, eps_rel, max_iter, polish, etc.)
        """
        self.spec = spec
        self.nz = spec.nU + spec.nS
        self._model = osqp.OSQP()
        # Initial dummy vectors (will be updated before the first solve)
        q0 = np.zeros(self.nz)
        l0 = -np.inf * np.ones(spec.A.shape[0])
        u0 =  np.inf * np.ones(spec.A.shape[0])

        settings = dict(
            eps_abs=1e-4,
            eps_rel=1e-4,
            max_iter=4000,
            polish=False,
            verbose=False,
            adaptive_rho=True,
        )
        if osqp_settings:
            settings.update(osqp_settings)

        self._model.setup(P=spec.P, q=q0, A=spec.A, l=l0, u=u0, **settings)

        # Warm-start cache
        self._z_prev = np.zeros(self.nz)
        self._solve_time = 0.0
        self._last_status = None

    @staticmethod
    def _shift_sequence(vec, block=1):
        """Shift a stacked sequence by one block, repeating the last block."""
        vec = vec.copy()
        if vec.size % block != 0:
            raise ValueError("Vector size must be multiple of block size")
        T = vec.size // block
        if T <= 1:
            return vec
        head = vec[block: T*block]
        tail = vec[(T-1)*block: T*block]
        return np.concatenate([head, tail])

    def _shift_warm_start(self, z):
        """Shift U (and s if present) by one step for warm-start."""
        if self.spec.use_soft_angle:
            U = z[:self.spec.nU]
            S = z[self.spec.nU:]
            U2 = self._shift_sequence(U, block=self.spec.n_u)  # n_u=1 here, but keep general
            S2 = self._shift_sequence(S, block=1)
            return np.concatenate([U2, S2])
        else:
            return self._shift_sequence(z, block=self.spec.n_u)

    def solve(self, x0, x_refs=None, u_refs=None, uref_scalar=0.0):
        """
        One MPC step: update vectors with x0, warm-start, solve, and return u0 and diagnostic info.
        Returns:
          u0: scalar control to apply
          z_star: optimal decision vector [U; s?]
          info: dict with keys {'solve_time', 'status', 'obj_val'}
        """
        q, l, u = self.spec.build_vectors(x0, x_refs=x_refs, u_refs=u_refs, uref_scalar=uref_scalar)

        # Update vectors in OSQP and warm-start
        self._model.update(q=q, l=l, u=u)
        z_warm = self._shift_warm_start(self._z_prev)
        self._model.warm_start(x=z_warm)

        t0 = time.perf_counter()
        res = self._model.solve()
        t1 = time.perf_counter()
        self._solve_time = (t1 - t0)
        self._last_status = res.info.status

        if res.x is None:
            # Fallback if solver fails (keep last input)
            z_star = self._z_prev
            u0 = float(z_star[0]) if z_star.size > 0 else 0.0
            status = f"fail({self._last_status})"
            obj = np.nan
        else:
            z_star = res.x.copy()
            u0 = float(z_star[0])
            status = res.info.status
            obj = res.info.obj_val

        # Cache for next warm start
        self._z_prev = z_star.copy()

        info = dict(solve_time=self._solve_time, status=status, obj_val=obj)
        return u0, z_star, info

# Module 4 — Real-time Simulation

### 4.1 Setup and goals

We simulate a nonlinear frictionless cart–pole (Module 1) controlled by our linear MPC (Modules 2–3). At $t=0$ the plant is at the upright equilibrium; at $t\approx 0.5 s$ we inject a small disturbance to the pole angle. The controller runs every $\Delta t$ seconds with horizon $N$.

### 4.2 Concurrency and timing
* Thread A (Controller / OSQP):
waits for the latest measured state $x_k$, builds ($q,\ell,u$), warm-starts, solves OSQP, and publishes the first control $u_k$.
* Thread B (Physics + Rendering) (main thread):
at fixed period $\Delta t$, reads the latest $u_k$ (hold-last if the solver overruns), integrates the nonlinear dynamics via RK4, logs, and re-draws with Matplotlib blitting.

Real-time discipline: the physics/render loop maintains a wall-clock period $\Delta t$ using deadlines; if the solver reply is late, we reuse the previous control for that step (standard “zero-order hold”).

### 4.3 Disturbance

At a chosen time $t_d$ (default $0.5 s$) we kick the pole slightly, e.g.,
$$\theta \leftarrow \theta + 0.035\ \text{rad} \ (\approx 2^\circ), \quad \dot\theta \leftarrow \dot\theta + 0.3\ \text{rad/s}.
$$
You can also randomize or repeat disturbances.

⸻

### 4.4 Module 4 — Implementation (Code cell)

In [ ]:
def rk4_step(f, x, u, dt, par):
    """One RK4 step for xdot = f(x,u)."""
    k1 = f(x, u, par)
    k2 = f(x + 0.5*dt*k1, u, par)
    k3 = f(x + 0.5*dt*k2, u, par)
    k4 = f(x + dt*k3, u, par)
    return x + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch
from matplotlib.lines import Line2D

class CartPoleRenderer:
    """Update artists + draw_idle; no blitting, no display handles, no auto-show."""
    def __init__(self, params, xlim=(-2.6, 2.6), dpi=110, u_max=10.0):
        self.par = params
        self.u_max = float(u_max)

        self.fig, self.ax = plt.subplots(figsize=(6.8, 2.6), dpi=dpi)
        self.ax.set_xlim(*xlim)
        self.ax.set_ylim(-params.l*2.2, params.l*2.2)
        self.ax.set_aspect('equal', adjustable='box')
        self.ax.set_xticks([]); self.ax.set_yticks([])
        for s in ("top","right","left","bottom"):
            self.ax.spines[s].set_visible(False)

        # Track
        self.track = Line2D([xlim[0], xlim[1]], [0, 0], lw=1.0, alpha=0.8)
        self.ax.add_line(self.track)

        # Cart
        self.cart_width = 0.30
        self.cart_height = 0.12
        self.cart = Rectangle((0 - self.cart_width/2, -self.cart_height/2),
                              self.cart_width, self.cart_height,
                              linewidth=1.0, edgecolor='k', facecolor='white')
        self.ax.add_patch(self.cart)

        # Pole
        self.pole = Line2D([0, 0], [0, 0], lw=1.8, color='k', alpha=0.9)
        self.ax.add_line(self.pole)

        # Control arrow (u)
        self.force_arrow = FancyArrowPatch((0, 0), (0, 0),
                                           arrowstyle='-|>', mutation_scale=16,
                                           lw=1.8, color='C0', alpha=0.9)
        self.ax.add_patch(self.force_arrow)

    def _cart_anchor(self, p):
        return (p, self.cart_height/2.0)

    def _update_force_arrow(self, p, u):
        base_x, base_y = p, 0.0
        Lnorm = np.clip(u / self.u_max, -1.0, 1.0) if self.u_max > 0 else 0.0
        dx = 0.75 * Lnorm
        self.force_arrow.set_positions((base_x, base_y), (base_x + dx, base_y))

    def draw_state(self, x, u=0.0):
        p, _, th, _ = x
        # Cart
        self.cart.set_x(p - self.cart_width/2)
        # Pole
        hx, hy = self._cart_anchor(p)
        L = 2.0 * self.par.l
        px = hx + L * math.sin(th)
        py = hy + L * math.cos(th)
        self.pole.set_data([hx, px], [hy, py])
        # Arrow
        self._update_force_arrow(p, u)

    def draw(self):
        import sys
        c = self.fig.canvas
        # In IPython/Jupyter, draw_idle is efficient; in plain scripts, force a full draw.
        if "ipykernel" in sys.modules:
            c.draw_idle()
        else:
            try:
                c.draw()  # explicit, synchronous draw for MacOSX/Tk/Qt in scripts
            except Exception:
                c.draw_idle()
        try:
            c.flush_events()
        except Exception:
            pass
        try:
            import matplotlib.pyplot as plt
            plt.pause(0.001)  # yield to GUI event loop
        except Exception:
            pass

In [ ]:
def build_mpc_for_cartpole(par: CartPoleParams, dt=DT, N=N_HORIZON, soft_angle=True):
    # Linearize & discretize
    A, B, _, _ = build_discrete_linear_cartpole(par, dt)
    # Weights and constraints
    Q, R, Qf = cartpole_default_weights()

    constraints = ConstraintConfig()
    constraints.u_max = 10.0
    constraints.x_max = 2.4
    constraints.soft_angle.enabled = bool(soft_angle)
    if constraints.soft_angle.enabled:
        constraints.soft_angle.theta_max = 0.25
        constraints.soft_angle.lambda_s  = 100.0

    spec = prepare_cartpole_qp_structure(A, B, Q, R, Qf, N, constraints)
    # mpc = LinearMPC(spec, osqp_settings=dict(eps_abs=1e-4, eps_rel=1e-4, max_iter=4000, verbose=False))
    mpc = LinearMPC(spec, osqp_settings=dict(eps_abs=3e-4, eps_rel=3e-4, max_iter=2000, verbose=False, adaptive_rho=True))
    return mpc, (A, B, Q, R, Qf, constraints)

In [ ]:
import time, threading
import numpy as np
import matplotlib.pyplot as plt

class RealtimeMPCSimulator:
    def __init__(self, par, mpc, dt=DT, T=6.0,
                 disturb_time=0.5, disturb=(0.035, 0.3),
                 x0=None, u_max=10.0,
                 render_fps=24,         # 24–30 is smooth & light
                 phys_dt=None,          # physics substep, e.g. 0.005
                 use_threads=True,
                 close_old_figs=True):
        self.par = par
        self.mpc = mpc
        self.control_dt = float(dt)   # solver rate
        self.T = float(T)
        self.disturb_time = float(disturb_time)
        self.disturb = disturb
        self.x = np.zeros(4) if x0 is None else np.asarray(x0, dtype=float).ravel()
        self.u_latest = 0.0
        self.u_max = float(u_max)

        self.render_fps = int(render_fps)
        self.frame_dt = 1.0 / self.render_fps
        self.phys_dt = float(phys_dt) if phys_dt is not None else min(0.005, self.control_dt/2.0)
        self.use_threads = bool(use_threads)

        # dynamic logs (no index errors)
        self.t_log = []
        self.x_log = []
        self.u_log = []
        self.solve_log = []

        # one figure only
        if close_old_figs:
            try: plt.close('all')
            except Exception: pass
        self.renderer = CartPoleRenderer(par, xlim=(-2.6, 2.6), dpi=110, u_max=self.u_max)

        # shared for solver thread
        self._lock = threading.Lock()
        self._stop = threading.Event()
        self._last_solve_time = 0.0

    def _rk4(self, x, u, dt):
        return rk4_step(cartpole_rhs, x, u, dt, self.par)

    def _solver_loop(self):
        next_tick = time.monotonic()
        while not self._stop.is_set():
            now = time.monotonic()
            if now < next_tick:
                time.sleep(max(0.0, next_tick - now))
            next_tick += self.control_dt

            with self._lock:
                x_meas = self.x.copy()

            t0 = time.perf_counter()
            u0, _, info = self.mpc.solve(x_meas)
            t1 = time.perf_counter()

            with self._lock:
                self.u_latest = float(u0)
                # prefer OSQP's reported time; fallback to wall time
                self._last_solve_time = float(info.get('solve_time', t1 - t0))

    def run(self, show=True):
        # Ensure interactive updates for all backends
        try:
            import matplotlib.pyplot as plt
            plt.ion()
        except Exception:
            pass

        # start solver in background
        th = None
        if self.use_threads:
            th = threading.Thread(target=self._solver_loop, daemon=True)
            th.start()

        # show figure once (non-blocking)
        try:
            self.renderer.fig.canvas.draw_idle()
            plt.show(block=False)
        except Exception:
            pass

        # real-time loop: fixed FPS render; physics substeps; T seconds total
        sim_t = 0.0
        disturbed = False
        n_frames = int(np.ceil(self.T / self.frame_dt))
        deadline = time.monotonic()

        for _ in range(n_frames):
            # integrate physics from sim_t to sim_t + frame_dt in fixed substeps
            remaining = self.frame_dt
            while remaining > 1e-12:
                dt_sub = min(self.phys_dt, remaining)
                with self._lock:
                    u = self.u_latest  # ZOH
                self.x = self._rk4(self.x, u, dt_sub)
                sim_t += dt_sub
                remaining -= dt_sub
                if (not disturbed) and (sim_t >= self.disturb_time):
                    self.x[2] += self.disturb[0]
                    self.x[3] += self.disturb[1]
                    disturbed = True

            # log (append; no fixed-size arrays)
            self.t_log.append(sim_t)
            with self._lock:
                self.u_log.append(self.u_latest)
                self.solve_log.append(self._last_solve_time)
            self.x_log.append(self.x.copy())

            # draw & give UI a timeslice
            self.renderer.draw_state(self.x, u=self.u_log[-1])
            self.renderer.draw()
            try:
                import matplotlib.pyplot as plt
                plt.pause(self.frame_dt * 0.5)
            except Exception:
                pass
            # sleep to maintain FPS (best-effort in notebooks)
            deadline += self.frame_dt
            sleep_t = deadline - time.monotonic()
            if sleep_t > 0:
                time.sleep(sleep_t)
            else:
                # overrun; reset deadline to now to avoid drift
                deadline = time.monotonic()

        # stop solver thread cleanly
        self._stop.set()
        if th is not None:
            th.join(timeout=0.2)

        # return logs as arrays
        return dict(
            t=np.array(self.t_log, dtype=float),
            x=np.vstack(self.x_log).astype(float),
            u=np.array(self.u_log, dtype=float),
            solve_time=np.array(self.solve_log, dtype=float),
        )

# How to Use

* Please run the plain python version ```MPC.py``` to see the correct real-time animation.
* Please see Appendix for setting up virtual environment.

# Appendix: Running in a virtual environment in Cursor/VS

1. In your project folder (where the notebook lives), run in the terminal:

```bash
# Create venv in a subfolder called .venv
python3 -m venv .venv

# Activate it (Linux/macOS)
source .venv/bin/activate

# Activate it (Windows PowerShell)
.\.venv\Scripts\Activate.ps1
```
Now your prompt should show (.venv) at the beginning.

2. Install Jupyter kernel inside the venv

Still inside the activated venv:

```bash
pip install --upgrade pip
pip install jupyter ipykernel
```

Then register this venv as a kernel:

```bash
python -m ipykernel install --user --name=optimization_demo --display-name "Python (optimization_demo)"
```

Now in Cursor’s Jupyter notebook, you can switch the kernel to Python (optimization_demo).

3. Install project dependencies

For the modules we built (MPC + cart-pole simulation), you’ll need:
* numpy
* scipy
* matplotlib
* osqp

So inside the venv:

```bash
pip install numpy scipy matplotlib osqp
```

Or install everything in ```requirements.txt```

```bash
pip install -r requirements.txt
```